In [ ]:
%pip install ultralytics
%pip install roboflow

## Got the dataset from here:  
https://universe.roboflow.com/bachelor-thesis-ymaax/face-detection-operating-room/dataset/7/images?split=train

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="QsT5gRKZeRi12T24N74T")
project = rf.workspace("bachelor-thesis-ymaax").project("face-detection-operating-room")
version = project.version(7)
dataset = version.download("yolov8")

In [ ]:
print(dataset.location)

## You have to do this for whatever dumb reason

In [ ]:
import shutil
shutil.move("Face-Detection-(Operating-Room)-7/train", "Face-Detection-(Operating-Room)-7/Face-Detection-(Operating-Room)-7/train")
shutil.move("Face-Detection-(Operating-Room)-7/valid", "Face-Detection-(Operating-Room)-7/Face-Detection-(Operating-Room)-7/valid")

## Have to do this because "Face-Detection-(Operating-Room)-7" has parentheses  
*sigh*

In [ ]:
import os
data_yaml_path = os.path.join(dataset.location, "data.yaml")

## Before running this line, download the current data.yaml file and change this:  
test: ../test/images  
train: ../train/images  
val: ../valid/images  
<br>

## To this:  
test: Face-Detection-(Operating-Room)-7/test/images  
train: Face-Detection-(Operating-Room)-7/train/images  
val: Face-Detection-(Operating-Room)-7/valid/images  
<br>

## Or just upload the already modified data.yaml file from the local project directory

In [ ]:
!yolo task=detect mode=train model=yolov8m.pt data="{data_yaml_path}" epochs=100 batch=8 imgsz=640

## Why is it `data="{data_yaml_path}"` and not `data=data_yaml_path`?
<br>

The `"{data_yaml_path}"` is needed because we're using it in a shell command (note the `!` at the start of the line). Let me break this down:

```python
data_yaml_path = os.path.join(dataset.location, "data.yaml")  # This is a Python string

# These two lines would NOT work:
!yolo task=detect data=data_yaml_path  # Shell sees 'data_yaml_path' as literal text
!yolo task=detect data={data_yaml_path}  # Shell gets confused by unquoted { }

# This works because:
!yolo task=detect data="{data_yaml_path}"  # Quotes handle spaces/special chars in path
```

When we use `!` in Jupyter/Colab, it's passing the command to the shell (bash). The shell needs:
1. The entire path as one argument (quotes handle spaces and special characters)
2. The Python variable's value (not its name)

The curly brackets `{data_yaml_path}` tell Python to substitute the variable's value there, like an f-string, but it's actually part of IPython's syntax for variable substitution in shell commands.

If we were calling this in pure Python (like using `subprocess`), we wouldn't need the quotes and braces:
```python
import subprocess
subprocess.run(['yolo', 'task=detect', f'data={data_yaml_path}'])  # Pure Python way
```